# test_nominal_karl.py

In [3]:
from __future__ import print_function, division
import unittest
import numpy
import nominal_resolution


class KarlTest(unittest.TestCase):
    def generateGrid(self, nlon, nlat):
        nmax = nlon * nlat

        dlon = 360. / nlon
        dlat = 180. / (nlat)

        radius = 6371.
        blons = numpy.ma.zeros((nlon * nlat, 5))
        blats = numpy.ma.zeros((nlon * nlat, 5))
        rg = range(nlon * nlat)
        blons[:, 0] = [n % nlon + dlon / 2. for n in rg]
        blons[:, 1] = blons[:, 0]
        blons[:, 2] = numpy.ma.masked
        blons[:, 3] = [n % nlon - dlon / 2. for n in rg]
        blons[:, 4] = blons[:, 3]

        blats[:, 0] = [-90. + (n // nlon) * dlat for n in rg]
        blats[:, 1] = [-90. + (n // nlon + 1) * dlat for n in rg]
        blats[:, 2] = numpy.ma.masked
        blats[:, 3] = [-90. + (n // nlon + 1) * dlat for n in rg]
        blats[:, 4] = blats[:, 0]

        cellarea = radius**2 * (blons[:, 0] - blons[:, 3]) * numpy.pi / 180. * (
            numpy.sin(numpy.pi * blats[:, 3] / 180.) - numpy.sin(numpy.pi * blats[:, 0] / 180.))
        for i in numpy.arange(1, nlat + 1) * nlon - 1:
            cellarea[i] = numpy.ma.masked

        return cellarea, blats, blons, dlon, dlat

    def doit(self, nlon, nlat, resolution):
        print("Testing:", nlon, nlat)
        radius = 6371.
        cellarea, blats, blons, dlon, dlat = self.generateGrid(nlon, nlat)
        correct_resolution = radius * (dlat * numpy.pi / 180.) / 2. * (
            1. + ((dlat**2 + dlon**2) / (dlat * dlon)) * numpy.arctan(dlon / dlat))

        print("Correct: {:g}".format(correct_resolution))

        convertdeg2rad = True
        test_resolution = nominal_resolution.mean_resolution(
            cellarea, blats, blons, convertdeg2rad)

        print("Test resol: {:g}".format(test_resolution))

        print(
            "tol:",
            (test_resolution -
             correct_resolution) /
            correct_resolution)

        if nlon < 5:
            rtol = .05
        elif nlon < 25:
            rtol = .02
        else:
            rtol = 0.001
        print("rtol is:", rtol)
        self.assertTrue(
            numpy.allclose(
                correct_resolution,
                test_resolution,
                rtol))
        self.assertEqual(
            nominal_resolution.nominal_resolution(test_resolution),
            resolution)

    def testMultipleResolutions(self):
        self.doit(3, 4, "10000 km")
        self.doit(4, 3, "10000 km")
        self.doit(10, 9, "5000 km")
        self.doit(18, 90, "2500 km")
        self.doit(180, 10, "2500 km")
        self.doit(24, 30, "1000 km")
        self.doit(80, 40, "500 km")
        self.doit(180, 90, "250 km")
        self.doit(360, 180, "100 km")
        self.doit(900, 500, "50 km")
        self.doit(1800, 900, "25 km")
        self.doit(3600, 1800, "10 km")

print('END')

END


In [6]:
j=KarlTest()

j.testMultipleResolutions()

Testing: 3 4
Correct: 11725.3
Test resol: 11894.8
tol: 0.014460514545606266
rtol is: 0.05
Testing: 4 3
Correct: 10439.2
Test resol: 10007.5
tol: -0.041345613425669475
rtol is: 0.05
Testing: 10 9
Correct: 3898.05
Test resol: 3839.91
tol: -0.014916060617201684
rtol is: 0.02
Testing: 18 90
Correct: 1763.37
Test resol: 1766.15
tol: 0.0015755521734949257
rtol is: 0.02
Testing: 180 10
Correct: 2009.73
Test resol: 2009.52
tol: -0.0001020674550399417
rtol is: 0.001
Testing: 24 30
Correct: 1485.07
Test resol: 1482.25
tol: -0.0018961805036934144
rtol is: 0.02
Testing: 80 40
Correct: 643.184
Test resol: 642.959
tol: -0.00035001779594864233
rtol is: 0.001
Testing: 180 90
Correct: 285.86
Test resol: 285.84
tol: -6.908875156827598e-05
rtol is: 0.001
Testing: 360 180
Correct: 142.93
Test resol: 142.927
tol: -1.7269856787698505e-05
rtol is: 0.001
Testing: 900 500
Correct: 53.746
Test resol: 53.7458
tol: -2.5929480103258575e-06
rtol is: 0.001
Testing: 1800 900
Correct: 28.586
Test resol: 28.5859
tol: -

# test_nominal_mpas.py

In [9]:
from __future__ import print_function
import unittest
import numpy
import nominal_resolution

class MPASTest(unittest.TestCase):

    def testIt(self):
        radius = 6371.
        cellarea = numpy.array([1.,1.])
        blats= numpy.array([[-8.695716125630886, -8.743596230455271, -8.63516813909499, -8.47180192573253, -8.424977900462013, -8.532991604024021, -8.532991604024021], [83.0722451487354, 83.09827824557905, 83.24785526448542, 83.39810919356844, 83.37442734959542, 83.21842863955908, 83.21842863955908]])
        blons = numpy.array([[51.0650943464591, 51.21168642417699, 51.33863462019597, 51.30106434286725, 51.15568215435684, 51.028332877885546, 51.028332877885546], [123.28552104391477, 124.92199149446174, 125.46917766638191, 124.61458426939708, 122.93720368239897, 122.41045356078911, 122.41045356078911]])
        correct= numpy.array([35.959866777444184, 40.226179274733745])
        test_resolution, max_d = nominal_resolution.mean_resolution(cellarea, blats, blons, convertdeg2rad=True, returnMaxDistance=True )

        print("Test resol: {:g}".format(test_resolution))
        print("MAX:",max_d)

        self.assertTrue(numpy.allclose(max_d,correct))

j=MPASTest()

j.testIt()

Test resol: 38.093
MAX: [35.95986678 40.22617927]


In [29]:
from __future__ import print_function
import numpy as np
import nominal_resolution

lats = numpy.array([[8,10,10,8],[8,10,10,8]])
lons = numpy.array([[100,100,102,102],[102,102,104,104]])
area = numpy.array([1.,1.])

# lats = numpy.array([[8,10,10,8]])
# lons = numpy.array([[100,100,102,102]])
# area = numpy.array([1.])

print(lats.shape)
print(lons.shape)
print(area.shape)

res,max_d = nominal_resolution.mean_resolution(area, lats, lons, convertdeg2rad=True, returnMaxDistance=True)

print(res,max_d)

(2, 4)
(2, 4)
(2,)
312.56807406378624 [312.56807406 312.56807406]


In [59]:
'''
this determines ocean nominal resolution ("res") from grid spec file
'''

from __future__ import print_function
import numpy as np
import nominal_resolution
import netCDF4
import inspect


CRED = '\033[91m'
CEND = '\033[0m'

__file__='jupyter_notebook' #this can be deleted when written to a python script and loaded as module.

input_file='/OSM/CBR/OA_DCFP/work/col414/CAFEPP/g/data/p66/mac599/CMIP5/ancillary_files/grid_spec.auscom.20110618.nc'


ifh = netCDF4.Dataset(input_file, 'r')

wet = ifh.variables['wet'][:]

area_T = ifh.variables['area_T'][:] #units not important, only need to be relatively correct.
                                            
y_vert_T = ifh.variables['y_vert_T'][:]

x_vert_T = ifh.variables['x_vert_T'][:]

x_vert_T = np.moveaxis(x_vert_T, 0, -1)
y_vert_T = np.moveaxis(y_vert_T, 0, -1)

print('wet.shape=',wet.shape)
print('area_T.shape=',area_T.shape)
print('x_vert_T.shape=',x_vert_T.shape)
print('y_vert_T.shape=',y_vert_T.shape)

ii=300 #ok
jj=50 #ok

ii=0
jj=0

wet1d = np.reshape(wet,(300*360))

wet_points, = np.where(wet1d==1)

print('wet_points=',wet_points)

#raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

#np.moveaxis(x, 0, -1).shape

print(np.array([area_T[jj,ii]]), np.array([y_vert_T[jj,ii,:]]), np.array([x_vert_T[jj,ii,:]]))

res,max_d = nominal_resolution.mean_resolution(np.array(area_T[jj,ii]), np.array([y_vert_T[jj,ii,:]]), np.array([x_vert_T[jj,ii,:]]), convertdeg2rad=True, returnMaxDistance=True) #single point

print(CRED+'result='+CEND,res,max_d)

res,max_d = nominal_resolution.mean_resolution(np.reshape(area_T,(300*360)), np.reshape(y_vert_T,(300*360,4)), np.reshape(x_vert_T,(300*360,4)), convertdeg2rad=True, returnMaxDistance=True) #all global points

print(CRED+'result='+CEND,res,max_d)

area_T1d = np.reshape(area_T,(300*360))

y_vert_T1d = np.reshape(y_vert_T,(300*360,4))

x_vert_T1d = np.reshape(x_vert_T,(300*360,4))

area_T1d_wet = area_T1d[wet_points]

y_vert_T1d_wet = y_vert_T1d[wet_points,:]

x_vert_T1d_wet = x_vert_T1d[wet_points,:]

print('wet_points.shape=',wet_points.shape)
print('area_T1d_wet.shape=',area_T1d_wet.shape)
print('x_vert_T1d_wet.shape=',x_vert_T1d_wet.shape)
print('y_vert_T1d_wet.shape=',y_vert_T1d_wet.shape)

res,max_d = nominal_resolution.mean_resolution(area_T1d_wet, y_vert_T1d_wet, x_vert_T1d_wet, convertdeg2rad=True, returnMaxDistance=True) #only ocean points

print(CRED+'result='+CEND,res,max_d)

#therefore according to notes, >=72 and <160 means nominal resolution=100km

print('END')

wet.shape= (300, 360)
area_T.shape= (300, 360)
x_vert_T.shape= (300, 360, 4)
y_vert_T.shape= (300, 360, 4)
wet_points= [   444    445    455 ... 107977 107978 107979]
[6.40954519e+10] [[-78.         -78.         -77.75316839 -77.75316839]] [[-280. -279. -279. -280.]]
result= 36.036059939210496 [36.03605994]
result= 119.76607174638343 [36.03605994 36.03605994 36.03605994 ... 45.37774742 45.96307532
 46.6254488 ]
wet_points.shape= (69782,)
area_T1d_wet.shape= (69782,)
x_vert_T1d_wet.shape= (69782, 4)
y_vert_T1d_wet.shape= (69782, 4)
result= 121.77214367578752 [36.39421599 36.39421599 36.39421599 ... 44.20706228 43.99045495
 43.78760097]
END


In [74]:
'''
this determines atmosphere nominal resolution ("res") from file with lat,lat_bnds,lon_bnds
'''

from __future__ import print_function
import numpy as np
import nominal_resolution
import netCDF4
import inspect
import math

CRED = '\033[91m'
CEND = '\033[0m'

__file__='jupyter_notebook' #this can be deleted when written to a python script and loaded as module.

input_file='/OSM/CBR/OA_DCFP/work/col414/CAFEPP/CMIP6/CMIP6/CMIP/CSIRO/CAFE-1-0/historical/r1i1p1f1/day/tas/gn/v20171025/tas_day_historical_CAFE-1-0_r1i1p1f1_gn_20150101-20201231.nc'


ifh = netCDF4.Dataset(input_file, 'r')

lat_bnds = ifh.variables['lat_bnds'][:]
lon_bnds = ifh.variables['lon_bnds'][:]
lat = ifh.variables['lat'][:]
lon = ifh.variables['lon'][:]

nlat,=lat.shape
nlon,=lon.shape

print('nlat,nlon=',nlat,nlon)

print('lat_bnds.shape=',lat_bnds.shape)
print('lon_bnds.shape=',lon_bnds.shape)
print('lat.shape=',lat.shape)

rad = 4.0*math.atan(1.0)/180.0

clat = np.abs(np.cos(lat*rad))

print('clat.shape=',clat.shape)

area = np.zeros((nlat,nlon),dtype=float)

print('area.shape=',area.shape)

lat_vert = np.zeros((nlat,nlon,4),dtype=float)
lon_vert = np.zeros((nlat,nlon,4),dtype=float)

for j in range(nlat):
  for i in range(nlon):
    area[j,i] = clat[j]
    lat_vert[j,i,0] = lat_bnds[j,0]
    lat_vert[j,i,1] = lat_bnds[j,1]
    lat_vert[j,i,2] = lat_bnds[j,1]
    lat_vert[j,i,3] = lat_bnds[j,0]
    
    lon_vert[j,i,0] = lon_bnds[j,0]
    lon_vert[j,i,1] = lon_bnds[j,1]
    lon_vert[j,i,2] = lon_bnds[j,1]
    lon_vert[j,i,3] = lon_bnds[j,0]
    
res,max_d = nominal_resolution.mean_resolution(np.reshape(area,(nlat*nlon)), np.reshape(lat_vert,(nlat*nlon,4)), np.reshape(lon_vert,(nlat*nlon,4)), convertdeg2rad=True, returnMaxDistance=True) #only ocean points

print(CRED+'result='+CEND,res,max_d)

#therefore according to notes, >=72 and <160 means nominal resolution=100km

print('END')

nlat,nlon= 90 144
lat_bnds.shape= (90, 2)
lon_bnds.shape= (144, 2)
lat.shape= (90,)
clat.shape= (90,)
area.shape= (90, 144)
result= 317.14068248577416 [140.5553848 140.5553848 140.5553848 ... 140.5553848 140.5553848
 140.5553848]
END
